In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from pathlib import Path
import shutil
import numpy as np
import thor.data as data
import thor.data.dispatch as dispatch
import thor.grid as grid
import thor.track as track
import thor.option as option
import thor.visualize as visualize
import thor.analyze as analyze
import thor.option as option
import thor.attribute as attribute
import thor.utils as utils

notebook_name = "cpol_demo.ipynb"


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [24]:
# Parent directory for saving outputs
base_local = Path.home() / "THOR_output"
start = "2005-11-13T18:00:00"
end = "2005-11-13T20:00:00"

output_directory = base_local / "runs/cpol/cpol_demo_geographic"

if output_directory.exists():
    shutil.rmtree(output_directory)

output_directory.mkdir(parents=True, exist_ok=True)
options_directory = output_directory / "options"
options_directory.mkdir(parents=True, exist_ok=True)

times_dict = {"start": start, "end": end}
cpol_options = data.option.CPOLOptions(**times_dict)
era5_dict = {"latitude_range": [-14, -10], "longitude_range": [129, 133]}
era5_pl_options = data.option.ERA5Options(**times_dict, **era5_dict)
era5_dict.update({"data_format": "single-levels"})
era5_sl_options = data.option.ERA5Options(**times_dict, **era5_dict)
data_options = data.option.DataOptions(
    datasets=[cpol_options, era5_pl_options, era5_sl_options]
)
data_options.to_yaml(options_directory / "data.yml")

# altitude = list(np.arange(0, 20e3 + 500, 500))
# altitude = [float(alt) for alt in altitude]
# grid_options = grid.create_options(name="geographic", altitude=altitude)
# grid.check_options(grid_options)
# grid.save_grid_options(grid_options, options_directory)

grid_options = grid.GridOptions()
grid_options.to_yaml(options_directory / "grid.yml")

# Create the track_options dictionary
track_options = option.default_track_options(dataset="cpol")
track_options.to_yaml(options_directory / "track.yml")

# Create the display_options dictionary
visualize_options = {
    obj: visualize.option.runtime_options(obj, save=True, style="presentation")
    for obj in ["mcs"]
}
visualize_options["middle_echo"] = visualize.option.runtime_options(
    "middle_echo", save=True, style="presentation", figure_types=["mask"]
)
# visualize.option.save_display_options(visualize_options, options_directory)
visualize_options = None

2024-11-20 12:13:57,800 - thor.data.option - INFO - Generating cpol filepaths.
2024-11-20 12:13:57,804 - thor.data.option - INFO - Generating era5 filepaths.
2024-11-20 12:13:57,806 - thor.data.option - INFO - Generating era5 filepaths.
2024-11-20 12:13:57,828 - thor.grid - WARNING - altitude not specified. Using default altitudes.
2024-11-20 12:13:57,829 - thor.grid - WARNING - shape not specified. Will attempt to infer from input.


In [4]:
times = data.utils.generate_times(data_options.dataset_by_name("cpol"))
tracks = track.simultaneous_track(
    times,
    data_options,
    grid_options,
    track_options,
    visualize_options,
    output_directory=output_directory
)

2024-11-19 16:33:08,866 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/cpol/cpol_demo_geographic.
2024-11-19 16:33:08,867 - thor.track - INFO - Beginning simultaneous tracking.
2024-11-19 16:33:08,963 - thor.track - INFO - Processing 2005-11-13T18:00:08.
2024-11-19 16:33:08,965 - thor.data.aura - INFO - Updating cpol dataset for 2005-11-13T18:00:08.
2024-11-19 16:33:08,967 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.180000.nc
2024-11-19 16:33:09,798 - thor.track - INFO - Processing hierarchy level 0.
2024-11-19 16:33:09,799 - thor.track - INFO - Tracking convective.
2024-11-19 16:33:09,833 - thor.track - INFO - Tracking middle.
2024-11-19 16:33:09,836 - thor.track - INFO - Tracking anvil.
2024-11-19 16:33:09,840 - thor.track - INFO - Processing hierarchy level 1.
2024-11-19 16:33:09,841 - thor.track - INFO - Tracking mcs.
2024-11-19 16:33:09,864 - thor.match.match - INFO - Matching mcs objects.
2024-11-19 16:3

# Cartesian Coordinates

In [ ]:
output_directory = base_local / "runs/cpol/cpol_demo_cartesian"
options_directory = output_directory / "options"
options_directory.mkdir(parents=True, exist_ok=True)

if output_directory.exists():
    shutil.rmtree(output_directory)

# grid_options = grid.create_options(name="cartesian", regrid=False, altitude=altitude)
# grid.check_options(grid_options)
# grid.save_grid_options(grid_options, options_directory)

grid_options = grid.GridOptions(name="cartesian", regrid=False)
grid_options.to_yaml(options_directory / "grid.yml")
data_options.to_yaml(options_directory / "data.yml")
track_options.to_yaml(options_directory / "track.yml")

# visualize.option.save_display_options(visualize_options, options_directory)

times = data.utils.generate_times(data_options.dataset_by_name("cpol"))
tracks = track.simultaneous_track(
    times,
    data_options,
    grid_options,
    track_options,
    visualize_options,
    output_directory=output_directory,
)

2024-11-19 16:42:43,718 - thor.grid - WARNING - altitude not specified. Using default altitudes.
2024-11-19 16:42:43,718 - thor.grid - WARNING - shape not specified. Will attempt to infer from input.
2024-11-19 16:42:43,829 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/cpol/cpol_demo_cartesian.
2024-11-19 16:42:43,830 - thor.track - INFO - Beginning simultaneous tracking.
2024-11-19 16:42:43,912 - thor.track - INFO - Processing 2005-11-13T18:00:08.
2024-11-19 16:42:43,914 - thor.data.aura - INFO - Updating cpol dataset for 2005-11-13T18:00:08.
2024-11-19 16:42:43,915 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.180000.nc
2024-11-19 16:42:44,044 - thor.track - INFO - Processing hierarchy level 0.
2024-11-19 16:42:44,046 - thor.track - INFO - Tracking convective.
2024-11-19 16:42:44,096 - thor.track - INFO - Tracking middle.
2024-11-19 16:42:44,101 - thor.track - INFO - Tracking anvil.
2024-11-19 16:42:44,111 - 

# Analysis

In [16]:
analysis_options = analyze.mcs.AnalysisOptions()
analysis_options.to_yaml(options_directory / "analysis.yml")
# utils.save_options(analysis_options, filename="analysis", options_directory=output_directory / "options")
analyze.mcs.process_velocities(output_directory)
analyze.mcs.quality_control(output_directory, analysis_options)
analyze.mcs.classify_all(output_directory, analysis_options)

,,stratiform_offset,inflow,relative_stratiform_offset,tilt,propagation
time,universal_id,,,,,
2005-11-13 18:00:08,1,trailing,front,left,up-shear,shear-perpendicular
2005-11-13 18:10:23,1,trailing,left,left,up-shear,shear-perpendicular
2005-11-13 18:20:09,1,trailing,left,left,up-shear,shear-perpendicular
2005-11-13 18:30:09,1,trailing,left,left,up-shear,shear-perpendicular
2005-11-13 18:40:09,1,trailing,left,left,up-shear,shear-perpendicular
2005-11-13 18:50:08,1,trailing,left,left,up-shear,shear-perpendicular
2005-11-13 19:00:08,1,trailing,left,left,up-shear,shear-perpendicular
2005-11-13 19:10:23,1,trailing,left,left,up-shear,up-shear
2005-11-13 19:20:09,1,trailing,left,leading,up-shear,up-shear


In [22]:
figure_options = visualize.option.horizontal_attribute_options(
    "cpol_20051113", style="presentation", attributes=["velocity", "offset"]
)
start_time = np.datetime64("2005-11-13T18:00")
end_time = np.datetime64("2005-11-13T19:50")
visualize.attribute.mcs_series(output_directory, start_time, end_time, figure_options, parallel_figure=True, by_date=False)

2024-11-19 16:59:51,287 - thor.grid - WARNING - shape not specified. Will attempt to infer from input.
2024-11-19 16:59:51,558 - thor.visualize.attribute - INFO - Visualizing MCS at time 2005-11-13T18:00:08.000000000.
2024-11-19 16:59:51,562 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.180000.nc
2024-11-19 16:59:51,724 - thor.grid - WARNING - shape not specified. Will attempt to infer from input.
2024-11-19 16:59:52,181 - thor.visualize.attribute - INFO - Saving cpol_20051113 figure for 2005-11-13T18:00:08.000000000.



## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather 

2024-11-19 17:00:01,862 - thor.visualize.attribute - INFO - Visualizing MCS at time 2005-11-13T18:10:23.000000000.
2024-11-19 17:00:01,869 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.181000.nc
2024-11-19 17:00:01,980 - thor.visualize.attribute - INFO - Visualizing MCS at time 2005-11-13T18:20:09.000000000.
2024-11-19 17:00:01,983 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.182000.nc
2024-11-19 17:00:02,033 - thor.visualize.attribute - INFO - Visualizing MCS at time 2005-11-13T18:30:09.000000000.
2024-11-19 17:00:02,037 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.183000.nc
2024-11-19 17:00:02,054 - thor.visualize.attribute - INFO - Visualizing MCS at time 2005-11-13T18:40:09.000000000.
2024-11-19 17:00:02,060 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.184000.nc
2024-11-19 17:00:03,476 - thor.grid - WARNING - shape not specified. Will attemp